In [ ]:
%config Completer.use_jedi = False

In [ ]:
from scipy import stats
from scipy.stats import norm
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
# JS divergence
def KL_div(p_probs, q_probs):    
    KL_div = p_probs * np.log(p_probs / q_probs)
    return np.sum(KL_div)

def JS_Div(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    # normalize
    p = p/p.sum()
    q = q/q.sum()
    m = (p + q) / 2
    return (KL_div(p, m) + KL_div(q, m)) / 2

In [ ]:
def num_label_random(sets):
    df = pd.read_csv('../../data/weak_data/{}/annotation_random_label.csv'.format(sets))
    label_list = []
    for idx, row in df.iterrows():
        label_list += row['label'].split(' ')
    count_label = dict(Counter(label_list))
    return count_label

In [ ]:
def num_label_cluster(sets):
    df = pd.read_csv('../../data/weak_data/{}/Excel_anno.csv'.format(sets))
    label_list = []
    for idx, row in df.iterrows():
        if type(row['Label']) == type('a'):
            label_list.append(row['Label'])#.split(' ')
    count_label = dict(Counter(label_list))
    return count_label

In [ ]:
def num_label_full(sets):
    label_list = []
    with open('../../data/weak_data/{}/label.txt'.format(sets), 'r') as f:
        for line in f.readlines():
            label_list += line.strip().split(' ')
        f.close()
    count_label = dict(Counter(label_list))
    return count_label

In [ ]:
def retrieve_val(label_list, counter):
    num_list = []
    for i in label_list:
        if i in counter.keys():
            num_list.append(counter[i])
        else:
            num_list.append(1e-11)
            
    return num_list

In [ ]:
def gen_random_cluster_full_list(sets):
    random_counter = num_label_random(sets)
    cluster_counter = num_label_cluster(sets)
    full_counter = num_label_full(sets)
    
    full_med_label = list(full_counter.keys())
    
    rdm = retrieve_val(full_med_label, random_counter)
    clr = retrieve_val(full_med_label, cluster_counter)
    ful = retrieve_val(full_med_label, full_counter)
    
    return rdm, clr, ful

#### random selection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import davies_bouldin_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean
from sentence_transformers import SentenceTransformer

In [ ]:
def gen_sent_list(sets, num_samples):
    input_text_path = '../../data/weak_data/{}/full_text.txt'.format(sets)
    input_label_path = '../../data/weak_data/{}/label.txt'.format(sets)
    #model = SentenceTransformer('../../tools/sbert-uncased-finnish-paraphrase')
    sent_list = []
    with open(input_text_path, 'r') as f:
        for line in f.readlines():
            sent_list.append(line.strip())
        f.close()
    #sent_list = list(set(sent_list))
    
    label_list = []
    with open(input_label_path, 'r') as f:
        for line in f.readlines():
            label_list.append(line.strip())
        f.close()
    
    assert len(label_list) == len(sent_list)
    
    comb_idx_sent_list = []
    for i in range(len(sent_list)):
        item = (i, sent_list[i], label_list[i])
        comb_idx_sent_list.append(item)
    
    pos_list =  np.random.choice(list(range(len(comb_idx_sent_list))), num_samples)
    
    random_select_idx_sent_label = [comb_idx_sent_list[i] for i in pos_list]#np.random.choice(comb_idx_sent_list, num_samples)
    
    select_idx = [item[0] for item in random_select_idx_sent_label]
    
    select_sent = [item[1] for item in random_select_idx_sent_label]
    
    select_label = [item[2] for item in random_select_idx_sent_label]
    
    df = pd.DataFrame({'index': select_idx, 'sent': select_sent})
    df.to_csv('../../data/weak_data/{}/annotation_random_sent.csv'.format(sets), index=False)
    
    df = pd.DataFrame({'index': select_idx, 'label': select_label})
    df.to_csv('../../data/weak_data/{}/annotation_random_label.csv'.format(sets), index=False)
    
    #selected_label
    
    
    #return random_select_sent

In [ ]:
def label_sum(res):
    sum_list = []
    for (item1, item2) in zip(res[1:7], res[7:]):
        sum_list.append(item1+item2)
    return sum_list

In [ ]:
# Jensen-Shannon divergence --> smaller --> similar

#### KIR

In [ ]:
# KIR, LAH, OPER, RTG, SAD
# 225., 200., 198., 162., 200.
random = 0
cluster = 0
for i in range(10):
    sets = 'KIR'
    num_samples = 225
    gen_sent_list(sets, num_samples)
    res = gen_random_cluster_full_list(sets)
    result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
    result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
    random += result_random_full
    cluster += result_cluster_full
if random > cluster:
    print('{}: cluster better'.format(sets))
    print(random)
    print(cluster)

In [ ]:
# sets = 'KIR'
# res = gen_random_cluster_full_list(sets)
# result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
# result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
# if result_random_full > result_cluster_full:
#     print('{}: cluster better'.format(sets))
#     print(result_random_full)
#     print(result_cluster_full)

#### LAH

In [ ]:
# KIR, LAH, OPER, RTG, SAD
# 225., 200., 198., 162., 200.
random = 0
cluster = 0
for i in range(10):
    sets = 'LAH'
    num_samples = 200
    gen_sent_list(sets, num_samples)
    res = gen_random_cluster_full_list(sets)
    result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
    result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
    random += result_random_full
    cluster += result_cluster_full
if random > cluster:
    print('{}: cluster better'.format(sets))
    print(random)
    print(cluster)

In [ ]:
# sets = 'LAH'
# res = gen_random_cluster_full_list(sets)
# result_random_full= JS_Div(res[0] + label_sum(res[0]), res[0] + label_sum(res[2]))
# result_cluster_full= JS_Div(res[0] + label_sum(res[1]), res[0] + label_sum(res[2]))
# if result_random_full > result_cluster_full:
#     print('{}: cluster better'.format(sets))
#     print(result_random_full)
#     print(result_cluster_full)

#### OPER

In [ ]:
# KIR, LAH, OPER, RTG, SAD
# 225., 200., 198., 162., 200.
random = 0
cluster = 0
for i in range(10):
    sets = 'OPER'
    num_samples = 198
    gen_sent_list(sets, num_samples)
    res = gen_random_cluster_full_list(sets)
    result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
    result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
    random += result_random_full
    cluster += result_cluster_full
if random > cluster:
    print('{}: cluster better'.format(sets))
    print(random)
    print(cluster)

In [ ]:
# sets = 'OPER'
# res = gen_random_cluster_full_list(sets)
# result_random_full= JS_Div(res[0] + label_sum(res[0]), res[0] + label_sum(res[2]))
# result_cluster_full= JS_Div(res[0] + label_sum(res[1]), res[0] + label_sum(res[2]))
# if result_random_full > result_cluster_full:
#     print('{}: cluster better'.format(sets))
#     print(result_random_full)
#     print(result_cluster_full)

#### RTG

In [ ]:
# KIR, LAH, OPER, RTG, SAD
# 225., 200., 198., 162., 200.
random = 0
cluster = 0
for i in range(10):
    sets = 'RTG'
    num_samples = 162
    gen_sent_list(sets, num_samples)
    res = gen_random_cluster_full_list(sets)
    result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
    result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
    random += result_random_full
    cluster += result_cluster_full
if random > cluster:
    print('{}: cluster better'.format(sets))
    print(random)
    print(cluster)

In [ ]:
# sets = 'RTG'
# res = gen_random_cluster_full_list(sets)
# result_random_full= JS_Div(res[0] + label_sum(res[0]), res[0] + label_sum(res[2]))
# result_cluster_full= JS_Div(res[0] + label_sum(res[1]), res[0] + label_sum(res[2]))
# if result_random_full > result_cluster_full:
#     print('{}: cluster better'.format(sets))
#     print(result_random_full)
#     print(result_cluster_full)

#### SAD

In [ ]:
# KIR, LAH, OPER, RTG, SAD
# 225., 200., 198., 162., 200.
random = 0
cluster = 0
for i in range(10):
    sets = 'SAD'
    num_samples = 200
    gen_sent_list(sets, num_samples)
    res = gen_random_cluster_full_list(sets)
    result_random_full= JS_Div([res[0][0]] + label_sum(res[0]), [res[0][0]] + label_sum(res[2]))
    result_cluster_full= JS_Div([res[0][0]] + label_sum(res[1]), [res[0][0]] + label_sum(res[2]))
    random += result_random_full
    cluster += result_cluster_full
if random > cluster:
    print('{}: cluster better'.format(sets))
    print(random)
    print(cluster)

In [ ]:
# sets = 'SAD'
# res = gen_random_cluster_full_list(sets)
# result_random_full= JS_Div(label_sum(res[0]), label_sum(res[2]))
# result_cluster_full= JS_Div(label_sum(res[1]), label_sum(res[2]))
# if result_random_full > result_cluster_full:
#     print('{}: cluster better'.format(sets))
#     print(result_random_full)
#     print(result_cluster_full)